# Function Decorator and Closures

Function decorators let us "mark" functions in the source code hence their behavior in some way. This is powerful stuff, but mastering it requires understanding closures.

One of the newest reserved keywords in Python is `nolocal`, introduced in Python3.0. If you want to implement your own function decorators, you must know closures inside out, and then the need for `nonlocal` becomes obvious.

Aside from their application in decorators, closures also essential for effective aynchronous programming with callbacks, and for coding in a functional style whenever it makes sense.

The goal of this chapter is to explain exactly how function decorators work, from the simplest registration decorators to the rather more complicated parameterized ones. However, before we reach the goal we need to cover:

* How Python evaluates decorator syntax
* How Python decides whether a variable is local
* Why closures exist and how they work
* What problem is solved by nonlocal

With this grouding, we can trackle futher decorator topics:

* Implementing a well-behaved decorator
* Interesting decorators in the stand library
* Implementing a parameterized decorator

## Decorator 101

A decorator is a callable that takes another function as argument(the decorated function). The decorator may performs some processing with the decorated function, and returns it or replace it with another function or callable object.

Strictly speaking, decorators are just syntatic sugar. As we just saw, you can always simply call a decorator like any regular callable, passing another function.

To summarize: 
* The first crucial fact about decorators is that they have power to replace the decorated function with a different one. 
* **The second fact is that they are execute immediately when a moudle is loaded.**

## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined. That is usually at import time(i.e., when a module is loaded by Python).

Consider how decorators are commonly employed in real code, is unusual in two ways:

* The decorator function is defined in the same module as the decorated funtions. A real decorator defined in one module, and applied to functions in other modules
* The register decorator returns the same function passed  as argument. In practice, most decorators define an inner function and return it.

## Decorator-Enhanced Strategy Pattern

In [5]:
promos = []

def promotion(func):
    promos.append(func)
    return func

@promotion
def fidelity(order):
    pass

promos

[<function __main__.fidelity(order)>]

Most decorators do change the decorated function. They usually do it by defining an inner function and return it to replace the decorated function. Code that uses inner functions almost always depends on closures to operate corretly.

## Variable Scope Rules

In [19]:
b = 100

def f1(a):
    print(a)
    print(b)
    b = 10

f1(100)

100


UnboundLocalError: local variable 'b' referenced before assignment

**When Python compiles the body of the function, it decides that b is a local varaible because it is assigned within the function.** The generated bytecode reflects this decision and will try to fetch b from the local environment. Later, when the call f1(100) is made, the body of f2 fetches and prints the value of the local variable a, **but when trying to fetch the value of local variable b it discovers that b is unbounded.**

This is not a bug, but a design choice: **Python does not require you to declare variables, but assumes that a variable assigned in the body of a function is local. This is much better than the behavior of JavaScipt, which does not require variable declartions either, but if you do forget that a variable is local, you may cobber a gloabl variable without knowning.**

In [21]:
b = 100

def f2(a):
    global b
    print(a)
    print(b)
    b = 10

f2(100)

100
100


The `dis` provides an easy way to disassemble the bytecode of Python functions.

In [22]:
from dis import dis

In [23]:
dis(f1)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (10)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


> Load local name b. This show that compiler consider `b` a local variable, even if the assignment to `b` occurs later, because the nature of the raiable whether it is local or not - cannot change the body of the function.

In [24]:
dis(f2)

  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (10)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


* Load global name `print`
* Load local name `a`
* Load global name `b`

The CPython VM that runs the bytecode is a stack machine, so the operations LOAD and POP refer to stack.

## Closures

Actually, a closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there. It does not matter whether the function is anonymous or not; **what matters is that it can access nonglobal variables that are defined outside of its body.**

In [25]:
class Averager:
    
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        return sum(self.series)/len(self.series)

In [27]:
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [31]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        return sum(series) / len(series)
    
    return averager

In [32]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


![](https://processon.com/chart_image/5ed9e8cb6376891862196fd6.png)

The bindings for series is kept in the `__closure__` attributes of the returned function avg. Each item in `avg.__closure__` to a name in `avg.__code__.co_freevars`. These items are cells, and they have an attribute called `cell_content`

In [33]:
avg.__code__.co_varnames

('new_value',)

In [34]:
avg.__code__.co_freevars

('series',)

In [38]:
avg.__closure__

(<cell at 0x10f549708: list object at 0x10f18d588>,)

In [37]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [41]:
avg.__code__.co_stacksize

3

**To summarize: a closure is a function that retains the bindings of the free variables that exist when the function is defined, so that they can be used later when the function is invoked and the defining scope is no longer available.**

## The nonlocal Declaration

Our previous implementation of `make_averager` was not efficient. we stored all the values in the historical and computed their sum every time averager was called. A better would just store the total and the number of items, and compute the the mean from these two numbers.

In [48]:
def make_averager():
    total = 0
    count = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

UnboundLocalError: local variable 'count' referenced before assignment

The problem is that the statement `count += 1` actually means the same as `count = count + 1`, when `count` is a number or any immutable type. **So we are actually assigning to `count` in the body of averager, and that makes it a local variable.**

We did not have this problem before, because we never assigned to the `series` name; we only called `series.append` and invoked `sum` and `len` on it. So we took advantage of the fact that lists are mutable.

But with immutable types like numbers, strings, tuples, etc., all you do is read, but never update. If you try to rebind them, as in `count = count + 1`, then you are implicitly creating a local variable count. It is no longer a free variable, and therefor it not saved in the closure.

To work around this, the `nonlocal` declaration was introduced in Python3. It lets you flag a variable as a free variable even when it is assigned a new value within the function. If a new value is assigned to a nonlocal variable, the binding stored in the closure is changed.

In [50]:
def make_averager():
    total = 0
    count = 0
    
    def averager(new_value):
        nonlocal total, count
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


## Implementing a Simple Decorator

In [3]:
import time

In [2]:
def clock(func):
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        print('call %s(%s) [%.8fs]' % (func.__name__, ' '.join(str(arg) for arg in args), elapsed))
        
        return result
    
    return clocked

In [30]:
@clock
def factorial(n):
    return n * factorial(n-1) if n > 2 else 1

In [32]:
factorial(15)

call factorial(2) [0.00000058s]
call factorial(3) [0.00004363s]
call factorial(4) [0.00005924s]
call factorial(5) [0.00016306s]
call factorial(6) [0.00020468s]
call factorial(7) [0.00022145s]
call factorial(8) [0.00038424s]
call factorial(9) [0.00040620s]
call factorial(10) [0.00042098s]
call factorial(11) [0.00043818s]
call factorial(12) [0.00045339s]
call factorial(13) [0.00047141s]
call factorial(14) [0.00048551s]
call factorial(15) [0.00052185s]


653837184000

### How it works

In [33]:
@clock
def factorial(n):
    return n * factorial(n-1) if n > 2 else 1

Actually does this:

In [36]:
def factorial(n):
    return n * factorial(n-1) if n > 2 else 1

factorial = clock(factorial)
factorial

<function __main__.clock.<locals>.clocked(*args, **kwargs)>

`clock` gets the `factorial` function as its function argument. It then creates and returns the clocked function. The `factorial` now actually holds a reference to the clocked function. Each time `factorial(n)` is called, `clocked(n)` get executed.

This is a typical hehavior of a decorator: it replaces the decorated function with a new function that accepts the same argument and return whatever the decorated function was supposed to return, while doing some extra processing.

## Decorator in the Standard Library

Python has three built-in functions that designed to decorate mothods: `property`, `classmethod` and `staticmethod`. Another frequently seen decorator is `functools.wraps`, a helper for building well-behavior decorators. Two of the most interesting decorators is the standard library `lru_cache` and the brand-new `singledispatch`.

### Memoization with `functools.lru_cache`

A very practical decorator is `functools.lru_cache`. It implements memoization: an optimization technique that works by saving the results of provious invocations of an expensive function, avoid repeat computations on previously used arguments. The letters LRU stand for Least Recently Used, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

In [52]:
@clock
def fibnacci(n):
    if n < 2:
        return 1
    else:
        return fibnacci(n-1) + fibnacci(n-2)

In [53]:
%time fibnacci(10)

call fibnacci(1) [0.00000044s]
call fibnacci(0) [0.00000059s]
call fibnacci(2) [0.00022436s]
call fibnacci(1) [0.00000089s]
call fibnacci(3) [0.00031287s]
call fibnacci(1) [0.00000034s]
call fibnacci(0) [0.00000073s]
call fibnacci(2) [0.00008582s]
call fibnacci(4) [0.00043522s]
call fibnacci(1) [0.00000029s]
call fibnacci(0) [0.00000035s]
call fibnacci(2) [0.00003601s]
call fibnacci(1) [0.00000042s]
call fibnacci(3) [0.00007202s]
call fibnacci(5) [0.00054500s]
call fibnacci(1) [0.00000026s]
call fibnacci(0) [0.00000039s]
call fibnacci(2) [0.00003837s]
call fibnacci(1) [0.00000044s]
call fibnacci(3) [0.00007421s]
call fibnacci(1) [0.00000026s]
call fibnacci(0) [0.00000030s]
call fibnacci(2) [0.00003731s]
call fibnacci(4) [0.00014749s]
call fibnacci(6) [0.00072828s]
call fibnacci(1) [0.00000027s]
call fibnacci(0) [0.00000028s]
call fibnacci(2) [0.00003552s]
call fibnacci(1) [0.00000053s]
call fibnacci(3) [0.00007058s]
call fibnacci(1) [0.00000025s]
call fibnacci(0) [0.00000027s]
call fib

89

In [45]:
from functools import lru_cache

In [55]:

@lru_cache(2**10)
@clock
def fibnacci(n):
    if n < 2:
        return 1
    else:
        return fibnacci(n-1) + fibnacci(n-2)

In [56]:
%time fibnacci(10)

call fibnacci(1) [0.00000055s]
call fibnacci(0) [0.00000065s]
call fibnacci(2) [0.00023616s]
call fibnacci(3) [0.00031510s]
call fibnacci(4) [0.00034961s]
call fibnacci(5) [0.00037067s]
call fibnacci(6) [0.00039164s]
call fibnacci(7) [0.00041054s]
call fibnacci(8) [0.00043281s]
call fibnacci(9) [0.00044737s]
call fibnacci(10) [0.00046146s]
CPU times: user 493 µs, sys: 427 µs, total: 920 µs
Wall time: 514 µs


89

### Generic Function with Single Dispatch

In [58]:
import html


def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

In [60]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [61]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [62]:
htmlize(42)

'<pre>42</pre>'

Because we don't have method or function overloading in Python, we can't create variable of `htmlize` with different  signatures for each type we want to handle differently.

The new `functools.singledispatch` decorator in Python3.4 allows each module to contribute to the overall solution, and lets you easily provide specialized function even for classes that you can't edit. If you decorate a plain function with `@singledispatch`, it becomes a generic function: a group of functions to perform the same operation in different ways, depending on the type of the first argument.

In [65]:
import html
import numbers
import collections
from functools import singledispatch

In [67]:
@singledispatch
def htmlize(obj):
    content = html.escape(obj)
    return '<pre>{}</pre>'.format(content)

In [68]:
@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

In [69]:
@htmlize.register(numbers.Integral)
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

In [70]:
@htmlize.register(tuple)
@htmlize.register(collections.abc.MutableSequence)
def _(seq):
    inner = '<li>\n</li>'.join(htmlize(item) for item in seq)
    return '<ul>\n</ul>' + inner + '<li>\n</li>'

> Using ABCs for type checking allows your code to support existing or future classes that are either actual or virutal subclass of those ABCs.

A notable quality of the singledispatch mechanism is that you can register specialized functions anywhere in the system, in any module. If you later add a module with a new used-defined type, you can easily provide a new custom function to handle that type.

`singledispatch` is a well-thought-out addition to the standard library. The best documentation for it is [PEP443-Single-Dispatch generic function](https://www.python.org/dev/peps/pep-0443/)

> `@singledispatch` is not designed to bring JAVA-style method overloading to Python. A single class with many overload varations of a method is better than a single function with lengthy stretch of `if/elif/elif/elfi` block. The advantage of `@singledispatch` is supporting modular extension: each module can register a specialized function  for each type it supports.

## Stacked Decorators

In [1]:
from functools import lru_cache

In [4]:
@lru_cache(2**10)
@clock
def fibonacci(n):
    if n < 2:
        return 1

    return fibonacci(n-1) + fibonacci(n-2)

In [5]:
fibonacci(10)

call fibonacci(1) [0.00000044s]
call fibonacci(0) [0.00000045s]
call fibonacci(2) [0.00008113s]
call fibonacci(3) [0.00009889s]
call fibonacci(4) [0.00011608s]
call fibonacci(5) [0.00013359s]
call fibonacci(6) [0.00014942s]
call fibonacci(7) [0.00016673s]
call fibonacci(8) [0.00018296s]
call fibonacci(9) [0.00038154s]
call fibonacci(10) [0.00042548s]


89

## Parameterized Decorators

When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function. So how do you make a decorator accept other arguments? The answer is: make a decorator factory that takes those arguments and return a decorator, which is then applied to the function to be decorated.

### A Parameterized Registration Decorator

In [6]:
registry = set()

def register(active=True):
    def decorate(func):
        print('running register(active=%s)->decorate(%s)' % (active, func))
        
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    
    return decorate

In [8]:
@register(active=False)
def f1():
    print('running f1()')

running register(active=False)->decorate(<function f1 at 0x10ee2ad08>)


In [13]:
@register()
def f2():
    print('running f2()')

running register(active=True)->decorate(<function f2 at 0x10ec220d0>)


In [14]:
f2()

running f2()


### The Parameterized Clock Decorator

In [15]:
import time

In [28]:
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*args):
            t0 = time.time()
            _result = func(*args)
            elapsed = time.time() - t0
            name = func.__name__
            print(fmt.format(elapsed=elapsed, name=name, args=', '.join(repr(arg) for arg in args),
                             result=repr(_result)))
            return _result
        
        return clocked
    
    return decorate

In [34]:
@lru_cache(2**10)
@clock()
def fib(n):
    if n < 2:
        return 1
    return fib(n-1) + fib(n-2)

In [33]:
fib(10)

[0.00000119s] fib(1) -> 1
[0.00000000s] fib(0) -> 1
[0.00009322s] fib(2) -> 2
[0.00013304s] fib(3) -> 3
[0.00015092s] fib(4) -> 5
[0.00038600s] fib(5) -> 8
[0.00042105s] fib(6) -> 13
[0.00045395s] fib(7) -> 21
[0.00047374s] fib(8) -> 34
[0.00049305s] fib(9) -> 55
[0.00050998s] fib(10) -> 89


89

## Chapter Summary

We started with a simple `@register` decorator without a inner function, and finished with a parameterized `@clock` involving two level of nested functions.

Registration decorators, though simple in essence, have real applications in advanced Python frameworks. We applied the registration idea to an improvements of our strategy design pattern.

Paramerized decorators almost aways involve at least two nested functions, maybe more if you want to use `@functools.wraps` to produce a decorator that provides better support for more advanced techniques. One such technique is stacked decorators.

We also visited two awesome functions decorators provided in the functools module of standard library: `@lru_cache` and `@singledispatch`.

Understanding how decorators actually work required covering the difference between `import time` and `runtime`, then diving into variable scoping, closures, and the new `nonlocal` declaration. Mastering closures and nonlocal is valuable not only to build decorators, but also to code event-oriented programs for GUIs or asychronous I/O with callbacks.

## Futher Reading

**Python Decorators and the Decorator Design Pattern**

Python function decorator fit the general description of Decorator given by `Design Pattern`: "attatch additional responsibilites to an object dynamically. Decorator provide a flexible alternative to subclassing for exetending functionality."

At the implementation level, Python decorators do not resemble the classic Decorator design pattern, but an analogy can be made.

In the design pattern, Decorator and Component are abstract class. An instance of a concrete decorator wraps an instance of a concrete component in order to add behaviors to it. Quoting from *Design Pattern*:

```
The decorator conforms to the interface of the component it decorates so that its presence is transparent to the component's clients. The decorator forwards requests to the component and may perform additional actions before or after forwarding.
```

In Python, the decorator function plays the role of a concrete Decorator subclass, and the inner function it returns is a decorator instance. The returnd functions wraps the function to be decorated, which is analogous to the component in the design pattern. The returned function is transparent because it conforms to the interface of the component by accepting the same arguments. It forwards calls to the component and may perform additional actions either before or after it. Borrowing from the previous citation, we can adapt the last sentence to say that "Transparency lets you nest decorators recusively, thereby allowing an unlimited number of added behaviors." That is what enable stacked decorator to work.